# Extract pole-like street furniture

In this notebook we demonstrate how we can extract pole-like objects form labeled point clouds.

In this code we assume the point clouds have been labelled following the process in our [Urban PointCloud Processing](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/tree/main/datasets) project. For more information on the specifics of the datasets used, see [the description there](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/blob/main/datasets/README.md).

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pandas as pd
import os
import pathlib
import laspy
from tqdm import tqdm
from sklearn.cluster import DBSCAN 

#from upcp.labels import Labels # TODO change back and remove labels.py (after update wheel)
from labels import Labels
from upcp.utils import las_utils
from upcp.utils import ahn_utils
import upcp.utils.bgt_utils as bgt_utils
from upc_analysis import PoleExtractor 
from upc_analysis import visualization 

In [ ]:
# We provide some example data for demonstration purposes.
dataset_folder = '../datasets/pointcloud/' 
prefix = 'final_'
files = list(pathlib.Path(dataset_folder).glob(f'{prefix}*.laz'))

# Define the class we are interested in
target_label = 60  

# Labels used for ground points ('Road' and 'Other ground')
ground_labels = [1, 9]

# AHN data reader for elevation data. This is optional: the data is only used when
# the ground elevation cannot be determined from the labeled point cloud itself.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder, caching=False)

# BGT data reader for building shapes. This is optional: the data is used to check
# whether an extracted object is located within a building footprint. This might
# indicate a false positive.
# If an appropriate dataset is available, uncomment these lines to use it:
if os.path.exists('../datasets/bgt/bgt_buildings_oost.csv'):
    bgt_building_file = '../datasets/bgt/bgt_buildings_oost.csv'    
    bld_reader = bgt_utils.BGTPolyReader(bgt_building_file)
else:
    bld_reader = None

In [ ]:
# Settings for clustering
MIN_SAMPLES = 100
EPS = 0.6

In [ ]:
my_settings = str(MIN_SAMPLES) + '_' + str(int(EPS*10))
output_file = '../datasets/poles_extracted_' + my_settings + '.csv'

---
## Extracting pole-like objects

This method works by clustering points of a given target class, and then using statistics and PCA analysis on each cluster to determine the exact pole.

The result is a dataset with the following features for each extracted object:
```txt
rd_x, rd_y, z = X, Y, Z coordinates of the base of the pole
tx, ty, tz    = X, Y, Z coordinates of the top of the pole
height        = the height of the pole, in m
angle         = the angle of the pole, in degrees w.r.t. vertical
prob          = the average probability of the classification, if this data is available in the point cloud
n_points      = the number of points of the object
in_bld        = flag that indicates whether the object is located inside a building footprint
debug         = debug code, see below
tilecode      = tilecode in which the object was found
```
The debug code `A_B` indicates potential issues with either the ground elevation (A) or the pole extraction (B). A can be either 0 (no problems), 1 (no ground elevation found in the point cloud), or 2 (no ground elevation found in AHN). B can be either 0 (no problems), 3 (not enough data to determine the angle), or 4 (not enough data to determine the exact location).

In [ ]:
# Create PoleExtractor object
pole_extractor = PoleExtractor(target_label, ground_labels,
                              ahn_reader=ahn_reader, building_reader=bld_reader,
                               min_samples=MIN_SAMPLES, eps=EPS)   

In [ ]:
# Loop over point cloud files and extract objects
locations = []
for file in tqdm(files):
    tilecode = las_utils.get_tilecode_from_filename(file.as_posix())
    pc = laspy.read(file)
#    labels = pc.label
    labels = pc.final_label
    if 'probability' in pc.point_format.extra_dimension_names:
        probabilities = pc.probability
    else:
        probabilities = np.zeros_like(labels)

    if np.count_nonzero(labels == target_label) > 0:
        points = np.vstack((pc.x, pc.y, pc.z)).T
        tile_locations = pole_extractor.get_pole_locations(points, labels, probabilities, tilecode)
        locations.extend([(*x, tilecode) for x in tile_locations])

HEADERS = ['rd_x', 'rd_y', 'z', 'tx', 'ty', 'tz', 'height', 'angle', 'prob', 'n_points', 'in_bld', 'debug', 'tilecode']
poles_df = pd.DataFrame(locations, columns=HEADERS)

In [ ]:
# Save the data in CSV format.
poles_df.to_csv(output_file, index=False)

---
## Visualize extracted poles

In order to verify the results and spot potential issues, we can visualize individual objects. The visualization renders the object from two directions (X and Y axis), along with a 3D projection.

In [ ]:
# Optional: load data from CSV.
poles_df = pd.read_csv(output_file)

In [ ]:
len(poles_df)

#### Loop over all poles

In [ ]:
%matplotlib inline

In [ ]:
for row_id in range(0, len(poles_df)): 
    # Get the object features
    obj = poles_df.loc[row_id]
    obj_location = (obj.rd_x, obj.rd_y, obj.z)
    obj_top = (obj.tx, obj.ty, obj.tz)

    # Load the point cloud data for the tile containing this object
    cloud = laspy.read(f'{dataset_folder}{prefix}{obj.tilecode}.laz')
    points = np.vstack((cloud.x, cloud.y, cloud.z)).T
    labels = cloud.final_label
    colors = np.vstack((cloud.red, cloud.green, cloud.blue)).T / (2**16 - 1)

    # Get a mask for the point cloud around the object's location
    obj_mask = visualization.get_mask_for_obj(points, labels, target_label, obj_location, obj_top[2],
                                              obj_angle=obj.angle, min_samples=MIN_SAMPLES,
                                              eps=EPS, noise_filter=True)
    
    # Create path to store output image
    output_path = '../datasets/output/output_image_' + my_settings + '_' + str(row_id) + '.png'

    # Show the object and store
    visualization.plot_object(points[obj_mask], labels[obj_mask], colors=colors[obj_mask],
                              estimate=np.vstack((obj_location, obj_top)), output_path=output_path)

#### Look at one pole

In [ ]:
# Either choose a row number of sample one randomly
row_id = 0
# idx = np.random.randint(0, len(poles_df))
# row_id = poles_df.index[idx]

# Get the object features
obj = poles_df.loc[row_id]
obj_location = (obj.rd_x, obj.rd_y, obj.z)
obj_top = (obj.tx, obj.ty, obj.tz)

# Let's see the object in question
poles_df.loc[[row_id]]

In [ ]:
# Load the point cloud data for the tile containing this object
cloud = laspy.read(f'{dataset_folder}{prefix}{obj.tilecode}.laz')
points = np.vstack((cloud.x, cloud.y, cloud.z)).T
labels = cloud.final_label
colors = np.vstack((cloud.red, cloud.green, cloud.blue)).T / (2**16 - 1)

In [ ]:
# Get a mask for the point cloud around the object's location
obj_mask = visualization.get_mask_for_obj(points, labels, target_label, obj_location, obj_top[2],
                                          obj_angle=obj.angle, min_samples=MIN_SAMPLES,
                                          eps=EPS, noise_filter=True)

# Create path to store output image
output_path = '../datasets/output/output_image_' + my_settings + '_' + str(row_id) + '.png'

# Show the object and store
visualization.plot_object(points[obj_mask], labels[obj_mask], colors=colors[obj_mask],
                          estimate=np.vstack((obj_location, obj_top)), output_path=output_path)

In [ ]:
# Print location (to be pasted in Google Maps)
import geopandas as gpd
import shapely.geometry as sg
my_lon = float(poles_df.loc[[row_id]]['rd_x'])
my_lat = float(poles_df.loc[[row_id]]['rd_y'])
df_loc = gpd.GeoDataFrame({'geometry': [sg.Point(my_lon, my_lat)]}, crs='epsg:28992').to_crs("epsg:4326") 
print(float(df_loc['geometry'].y), float(df_loc['geometry'].x))